In [1]:
from keras.utils import to_categorical
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters
import networkx as nx

In [2]:
from sklearn.datasets import load_svmlight_file


def get_data(data):
    data = load_svmlight_file(data)
    return data[0].toarray(), data[1]
data = "./a1a.txt"
X, y = get_data(data)
y[y == -1] = 0
# Convert labels to one-hot encoding
y = to_categorical(y)

data = "./a1a_t"
X_test,y_test = get_data(data)
y_test[y_test == -1] = 0
y_test = to_categorical(y_test)

In [3]:
y[y[:, 0] == 1].shape
X.shape

(1605, 119)

In [4]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.shape[0])
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split

In [5]:
no_users = 30

In [6]:
X, y = random_split(X, y, 3, 42)

In [7]:
X1 = X[0][:, 0:50]
X2 = X[1][:, 30: 90]
X3 = X[2][:, 20:]
y1 = y[0]
y2 = y[1]
y3 = y[2]
subset_ranges = [np.arange(0, 50), np.arange(30, 90), np.arange(20, 119)]
subset_lengths = [subset_ranges[0].shape[0], subset_ranges[1].shape[0], subset_ranges[2].shape[0]]

In [8]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout, 5 * pout],[pout, pin, 5 * pout],[5 * pout, 5 * pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs, seed=0)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10, 10]
features_sizes = [8, 7, 6, 5]
pin = 0.5
pout = 0.1
seed = 0
alpha = 1e-2
lamda = 0#1e-2#1e-3
eta = 1e-2
d0 = min(subset_lengths)
no_users = sum(cluster_sizes)
batch_size = 10
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

# Set a random seed for reproducibility
seed = 17
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [9]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [10]:
y1.shape

(535, 2)

In [11]:
datapoints = {}
count = 0

X1, y1 = random_split(X1, y1, 10, 42)
X2, y2 = random_split(X2, y2, 10, 42)
X3, y3 = random_split(X3, y3, 10, 42)

X_train = [X1, X2, X3]
y_train = [y1, y2, y3]
input_sizes = [X1[0].shape[1], X2[0].shape[1], X3[0].shape[1]]

for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        
        test_features = X_test[:, subset_ranges[i]]
        test_label = y_test
        datapoints[count] = {
                'features': X_train[i][j],
                'degree': node_degree(count, G),
                'label': y_train[i][j],
                'neighbors': get_neighbors(count, G),
                'input_size': X_train[i][j].shape[1],
                'test_features':test_features,
                'test_labels': test_label
            }
        count += 1

In [12]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [13]:

class MLP_Net(nn.Module):
    def __init__(self, input_size, num_classes, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes, bias=True)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        output = F.softmax(self.fc1(x), dim=1)  # Applying softmax along the second dimension
        return output

In [14]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [15]:
model = MLP_Net(datapoints[0]["input_size"], 2, user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[0]["features"], datapoints[0]["label"]), batch_size=20, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(1000):
    for j, (x, y) in zip(range(1), dataloader):
        criterion = nn.CrossEntropyLoss()
        optimizer.zero_grad()
        yhat = model(x)
        
        # Calculate accuracy
        _, predicted = torch.max(yhat, 1)
        _, predicted_true = torch.max(y, 1)
        correct_predictions = (predicted == predicted_true).sum().item()
         

        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss.detach(), correct_predictions)
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(0.7243) 5
1 tensor(0.7168) 7
2 tensor(0.7072) 7
3 tensor(0.7037) 6
4 tensor(0.7381) 3
5 tensor(0.7233) 4
6 tensor(0.7124) 5
7 tensor(0.7149) 6
8 tensor(0.7011) 10
9 tensor(0.7044) 10
10 tensor(0.7113) 7
11 tensor(0.7167) 7
12 tensor(0.6863) 12
13 tensor(0.6950) 11
14 tensor(0.6973) 9
15 tensor(0.6853) 10
16 tensor(0.6938) 11
17 tensor(0.7039) 8
18 tensor(0.6882) 11
19 tensor(0.6918) 11
20 tensor(0.6919) 9
21 tensor(0.6995) 9
22 tensor(0.7058) 8
23 tensor(0.6918) 9
24 tensor(0.7008) 8
25 tensor(0.6934) 11
26 tensor(0.6974) 8
27 tensor(0.6901) 7
28 tensor(0.6792) 11
29 tensor(0.6794) 12
30 tensor(0.6853) 13
31 tensor(0.6725) 14
32 tensor(0.6809) 10
33 tensor(0.6903) 8
34 tensor(0.6936) 8
35 tensor(0.6877) 9
36 tensor(0.6837) 10
37 tensor(0.6899) 10
38 tensor(0.6755) 12
39 tensor(0.6671) 13
40 tensor(0.6928) 11
41 tensor(0.6868) 11
42 tensor(0.6961) 12
43 tensor(0.6730) 13
44 tensor(0.6791) 14
45 tensor(0.6768) 12
46 tensor(0.6698) 14
47 tensor(0.6851) 12
48 tensor(0.6689) 13
49 

391 tensor(0.5377) 16
392 tensor(0.5500) 15
393 tensor(0.5050) 17
394 tensor(0.5224) 17
395 tensor(0.5908) 13
396 tensor(0.5525) 16
397 tensor(0.5198) 17
398 tensor(0.5723) 15
399 tensor(0.5902) 14
400 tensor(0.5836) 14
401 tensor(0.5484) 15
402 tensor(0.5734) 14
403 tensor(0.5708) 15
404 tensor(0.5598) 15
405 tensor(0.6095) 13
406 tensor(0.5313) 16
407 tensor(0.5373) 17
408 tensor(0.5037) 17
409 tensor(0.5732) 14
410 tensor(0.5484) 15
411 tensor(0.5778) 14
412 tensor(0.5294) 16
413 tensor(0.4947) 18
414 tensor(0.5146) 16
415 tensor(0.4988) 17
416 tensor(0.5412) 16
417 tensor(0.5373) 16
418 tensor(0.5748) 14
419 tensor(0.5267) 16
420 tensor(0.5465) 16
421 tensor(0.5716) 14
422 tensor(0.5905) 14
423 tensor(0.6338) 12
424 tensor(0.5382) 16
425 tensor(0.5274) 17
426 tensor(0.5303) 16
427 tensor(0.5688) 15
428 tensor(0.5344) 16
429 tensor(0.5711) 14
430 tensor(0.5696) 14
431 tensor(0.5351) 16
432 tensor(0.5081) 16
433 tensor(0.5489) 15
434 tensor(0.4854) 18
435 tensor(0.5748) 14
436 tensor

770 tensor(0.5260) 16
771 tensor(0.4780) 18
772 tensor(0.4400) 19
773 tensor(0.4815) 17
774 tensor(0.4662) 18
775 tensor(0.5341) 16
776 tensor(0.5848) 13
777 tensor(0.5006) 16
778 tensor(0.6048) 13
779 tensor(0.4854) 17
780 tensor(0.4620) 18
781 tensor(0.5233) 16
782 tensor(0.5625) 14
783 tensor(0.5341) 15
784 tensor(0.5106) 16
785 tensor(0.4993) 17
786 tensor(0.5556) 14
787 tensor(0.5120) 16
788 tensor(0.5392) 16
789 tensor(0.4451) 19
790 tensor(0.4800) 18
791 tensor(0.5396) 15
792 tensor(0.4685) 17
793 tensor(0.5830) 13
794 tensor(0.5429) 15
795 tensor(0.5179) 17
796 tensor(0.5785) 13
797 tensor(0.4983) 17
798 tensor(0.5087) 16
799 tensor(0.5291) 16
800 tensor(0.5090) 16
801 tensor(0.5917) 13
802 tensor(0.5725) 14
803 tensor(0.5918) 13
804 tensor(0.5259) 15
805 tensor(0.5196) 15
806 tensor(0.5194) 16
807 tensor(0.5142) 16
808 tensor(0.5253) 16
809 tensor(0.5591) 14
810 tensor(0.5639) 15
811 tensor(0.5088) 16
812 tensor(0.5214) 16
813 tensor(0.5481) 15
814 tensor(0.5055) 15
815 tensor

In [16]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [17]:
# Preparing projection matrices
models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=2, user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, models, first_run=True):
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        # Specify the dimensions of the rectangular matrix
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]

                        # Generate random values for the diagonal from a normal distribution
                        diag_values = 1.0 + 1.0 * torch.randn(min(row, column))

                        # Create a rectangular matrix with diagonal elements
                        mat = torch.diag(diag_values)

                        # If the matrix is larger than the diagonal vector, fill the remaining elements with zeros
                       
                        mat = torch.cat((mat, torch.zeros(row, column - row)), dim=1)

                        

                        # Append the matrix to the list
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights, models)



In [18]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["test_features"], dataset["test_labels"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        _, predicted_true = torch.max(labels, 1)
        correct += pred.eq(predicted_true.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    correct /= len(test_loader.dataset)
    
    return test_loss, correct

In [19]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=2, user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=2, user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.CrossEntropyLoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

in_cluster_proj_norm = []
out_cluster_proj_norm = []
in_cluster_proj_diff_norm = []
out_cluster_proj_diff_norm = []
task_loss = {'0':[],
                '1':[],
                '2':[]}
task_rel_error = {'0':[],
            '1':[],
            '2':[]}

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    projected_weights = []
    update_ProjWeight(projection_list, projected_weights, models, first_run=False)
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            temp_mat = torch.outer(projected_weights[i][j] - projected_weights[j][i], weights).clone()
            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
    
    in_cluster_proj_norm_round = 0
    out_cluster_proj_norm_round = 0
    in_cluster_proj_diff_round = 0
    out_cluster_proj_diff_round = 0
    in_edges = 0
    out_edges = 0
    
    for i in range(no_users//3):
        for j in G.neighbors(i):
            if j < no_users//3:
                in_edges += 1
                in_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                in_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
            else:
                out_edges += 1
                out_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                out_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
    in_cluster_proj_norm.append(in_cluster_proj_norm_round / in_edges)
    out_cluster_proj_norm.append(out_cluster_proj_norm_round / out_edges)
    in_cluster_proj_diff_norm.append(in_cluster_proj_diff_round / in_edges)
    out_cluster_proj_diff_norm.append(out_cluster_proj_diff_round / out_edges)
            
                                         
                                              
    
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    per_task_loss = []
    per_task_rel_error = []
    
    for k in range(no_users):
      
        g_loss, acc = testing(models[k], datapoints[k], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += acc#rel_error(models[i])
        if (k + 1) % 10 == 0:
            task_loss[str(k // 10)].append(sum(per_task_loss) / 10)
            task_rel_error[str(k // 10)].append(sum(per_task_rel_error) / 10)
            per_task_loss = []
            per_task_rel_error = []
        per_task_loss.append(g_loss)
        per_task_rel_error.append(acc)#rel_error(models[i]))
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Accuracy %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 1/2000 [00:18<10:04:17, 18.14s/it]

Training_loss 0.69400,   Accuracy 0.49996


  0%|          | 2/2000 [00:39<11:08:25, 20.07s/it]

Training_loss 0.69253,   Accuracy 0.50915


  0%|          | 3/2000 [00:57<10:32:41, 19.01s/it]

...

Training_loss 0.54779,   Accuracy 0.77113


 33%|███▎      | 658/2000 [15:21:21<6:58:32, 18.71s/it]

Training_loss 0.54778,   Accuracy 0.77130


 33%|███▎      | 658/2000 [15:21:29<31:19:23, 84.03s/it]


KeyboardInterrupt: 

In [20]:
#plot.plot(test_loss)
print(parameters_to_vector(models[19].parameters()), W[1])

NameError: name 'W' is not defined

In [21]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

in_cluster_proj_norm = np.array(in_cluster_proj_norm)
out_cluster_proj_norm = np.array(out_cluster_proj_norm)
in_cluster_proj_diff_norm = np.array(in_cluster_proj_diff_norm)
out_cluster_proj_diff_norm = np.array(out_cluster_proj_diff_norm)


In [22]:
'''
  0%|          | 1/2000 [00:12<6:59:49, 12.60s/it]
Training_loss 0.69317,   Accuracy 0.52177
  0%|          | 2/2000 [00:25<6:58:41, 12.57s/it]
Training_loss 0.69256,   Accuracy 0.52523
  0%|          | 3/2000 [00:37<6:55:30, 12.48s/it]
Training_loss 0.69282,   Accuracy 0.52400
  0%|          | 4/2000 [00:50<6:56:56, 12.53s/it]
Training_loss 0.69216,   Accuracy 0.52552
  0%|          | 5/2000 [01:05<7:35:34, 13.70s/it]
Training_loss 0.69178,   Accuracy 0.52765
  0%|          | 6/2000 [01:18<7:24:07, 13.36s/it]
Training_loss 0.69037,   Accuracy 0.55240
  0%|          | 7/2000 [01:32<7:32:02, 13.61s/it]
Training_loss 0.68986,   Accuracy 0.55653
  0%|          | 8/2000 [01:47<7:42:39, 13.94s/it]
Training_loss 0.68921,   Accuracy 0.56338
'''

'\n  0%|          | 1/2000 [00:12<6:59:49, 12.60s/it]\nTraining_loss 0.69317,   Accuracy 0.52177\n  0%|          | 2/2000 [00:25<6:58:41, 12.57s/it]\nTraining_loss 0.69256,   Accuracy 0.52523\n  0%|          | 3/2000 [00:37<6:55:30, 12.48s/it]\nTraining_loss 0.69282,   Accuracy 0.52400\n  0%|          | 4/2000 [00:50<6:56:56, 12.53s/it]\nTraining_loss 0.69216,   Accuracy 0.52552\n  0%|          | 5/2000 [01:05<7:35:34, 13.70s/it]\nTraining_loss 0.69178,   Accuracy 0.52765\n  0%|          | 6/2000 [01:18<7:24:07, 13.36s/it]\nTraining_loss 0.69037,   Accuracy 0.55240\n  0%|          | 7/2000 [01:32<7:32:02, 13.61s/it]\nTraining_loss 0.68986,   Accuracy 0.55653\n  0%|          | 8/2000 [01:47<7:42:39, 13.94s/it]\nTraining_loss 0.68921,   Accuracy 0.56338\n'

In [23]:
np.save( 'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)
np.save( 'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_norm)
np.save('out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_norm)
np.save( 'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_diff_norm)
np.save('out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_diff_norm)

In [24]:
# task_loss dictionary
for key, value in task_loss.items():
    # Convert the list to a NumPy array
    array_loss = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('training_loss_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_loss)

# task_rel_error dictionary
for key, value in task_rel_error.items():
    # Convert the list to a NumPy array
    array_rel_error = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_rel_error)